# Data Cleaning and Preparation

In this section, we will clean and prepare the dataset for analysis by following the steps outlined in the book *Python for Data Analysis* by Wes McKinney. Data cleaning is a crucial step to ensure the quality of the dataset before performing any analysis.

## Steps for Data Cleaning

1. **Loading the Dataset**
   - Load the dataset into a pandas DataFrame using `pd.read_csv()` or another appropriate function.

2. **Handling Missing Data**
   - Identify missing data with `isna()` or `isnull()`.
   - Handle missing data using:
     - `dropna()` to remove missing values.
     - `fillna()` to impute missing values.

3. **Removing Duplicates**
   - Use the `drop_duplicates()` method to remove duplicate rows from the dataset.

4. **Data Transformation**
   - **Renaming Columns**: Rename columns with `rename()`.
   - **Replacing Values**: Replace values using `replace()`.
   - **Applying Functions**: Apply functions to columns using `apply()` or `map()`.
   - **Binning/Discretization**: Discretize continuous data into bins with `cut()` or `qcut()`.
   - **Handling Outliers**: Filter outliers using conditional filtering or z-scores.
   - **Random Sampling**: Randomly sample data with `sample()`.
   - **Indicator Variables**: Create dummy variables using `get_dummies()`.

5. **String Manipulation**
   - Use pandas' string methods accessed with `.str` to manipulate strings efficiently.
   - Apply regular expressions for complex string operations with `str.contains()`, `str.replace()`, etc.

6. **Categorical Data**
   - Convert categorical variables to pandas' `Categorical` type for efficient memory use and performance.
   - Use `Categorical` methods like `categories`, `ordered`, and `codes`.



# Step 1: Loading the Dataset

In this step, we will load the dataset into a pandas DataFrame for further cleaning and analysis. Pandas provides various methods for reading data from different formats such as CSV, Excel, and SQL databases. 

We will use `pd.read_csv()` to load a CSV file, but this method can easily be adapted for other formats.

Let's start by loading the dataset and inspecting the first few rows to understand its structure.

In [7]:
### Code Cell:
import pandas as pd

# Load the dataset into a pandas DataFrame
df = pd.read_csv('loandata.csv')

# Display the first few rows of the dataset to inspect its structure
df.head()


,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001722,Male,Yes,0,Graduate,No,150,1800.0,135.0,360.0,1.0,Rural,N
1,LP002502,Female,Yes,2,Not Graduate,NaN,210,2917.0,98.0,360.0,1.0,Semiurban,Y
2,LP002949,Female,No,3+,Graduate,NaN,416,41667.0,350.0,180.0,NaN,Urban,N
3,LP002603,Female,No,0,Graduate,No,645,3683.0,113.0,480.0,1.0,Rural,Y
4,LP001644,NaN,Yes,0,Graduate,Yes,674,5296.0,168.0,360.0,1.0,Rural,Y


In [11]:
print("\nChecking for Missing Values:")
missing_values = df.isnull().sum()
display(missing_values[missing_values > 0])


Checking for Missing Values:


Gender              13
Married              3
Dependents          15
Self_Employed       32
LoanAmount          22
Loan_Amount_Term    14
Credit_History      50
dtype: int64

## Key Insights:
- The **Credit_History** column has the highest number of missing values (50), which may significantly impact analysis, especially if it is a crucial feature in predicting loan approval.
- Other columns like **Self_Employed**, **LoanAmount**, and **Dependents** also have a considerable amount of missing data, which may need to be handled carefully depending on their relevance.
- **Married** and **Gender** have fewer missing values, making them relatively easier to handle.

## Next Steps:
1. **Assess the importance of each column**: We should determine how critical each column is to our analysis or model before deciding how to handle the missing values.
2. **Handling missing data**:
   - For **categorical columns** (e.g., **Gender**, **Married**, **Self_Employed**), consider imputation with the mode or use a placeholder value like `'Unknown'`.
   - For **numerical columns** (e.g., **LoanAmount**, **Loan_Amount_Term**), consider imputation with the median or a statistical method like regression imputation.
3. **Explore potential relationships**: Investigate whether there is any pattern or relationship between missing values and other features in the dataset.
4. **Decide on deletion or imputation**: Based on the amount of missing data and the importance of each feature, decide whether to drop rows or impute missing values.

Next, we'll proceed with handling the missing values using the appropriate techniques.

# Handling Missing Values

In this step, we will handle the missing values in the dataset. For numerical columns, we will impute missing values with the **median**, and for non-numerical columns, we will impute missing values with the **mode** (most frequent value).

### Plan:
- **Numerical Columns**: Impute missing values using the median (which is more robust to outliers).
- **Categorical Columns**: Impute missing values using the mode (most frequent value).


In [15]:
# Impute missing values for numerical columns with the median
numerical_cols = ['LoanAmount', 'Loan_Amount_Term']  # Add other numerical columns here if needed
for col in numerical_cols:
    df[col].fillna(df[col].median(), inplace=True)

# Impute missing values for categorical columns with the mode
categorical_cols = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'Credit_History']  # Add other categorical columns here if needed
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Display the number of missing values after imputation
df.isna().sum()


C:\Users\santo\AppData\Local\Temp\ipykernel_13272\1762916487.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\santo\AppData\Local\Temp\ipykernel_13272\1762916487.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

# Step 3: Removing Duplicates

Duplicate rows in a dataset can lead to biased analysis and inaccurate results. In this step, we will check for duplicate entries and remove them if necessary.

## Approach:
- Use `df.duplicated().sum()` to count the number of duplicate rows.
- Use `df.drop_duplicates(inplace=True)` to remove duplicate rows while keeping only the first occurrence.
- Recheck the dataset to confirm that duplicates have been removed.

In [21]:
# Check for duplicate rows in the dataset
duplicate_count = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicate_count}")

# Remove duplicate rows
if duplicate_count > 0:
    df = df.drop_duplicates(inplace=True)
    print("Duplicates removed. Dataset now contains")
else:
    print("No Duplictes found. Dataset contains")

# Verify that duplicates have been removed
print(f"Number of duplicate rows after removal: {df.duplicated().sum()}")

# Display dataset shape after removing duplicates
df.shape

Number of duplicate rows: 0
No Duplictes found. Dataset contains
Number of duplicate rows after removal: 0


(614, 13)

# Step 4: Data Transformation

Data transformation is an essential step in cleaning and preparing the dataset for analysis. It involves modifying the dataset’s structure and values to improve consistency, accuracy, and usability.

We will perform the following transformations, each in a separate step:

1. **Renaming Columns**: Standardizing column names for better readability.
2. **Replacing Values**: Handling inconsistencies in categorical variables.
3. **Applying Functions**: Applying transformations or calculations to specific columns.
4. **Binning/Discretization**: Converting continuous numerical data into categories.
5. **Handling Outliers**: Identifying and treating outliers.
6. **Random Sampling**: Creating a subset of the dataset for analysis.
7. **Creating Dummy Variables**: Converting categorical variables into numeric form.

## Renaming Columns

To maintain consistency and readability, we will rename columns where necessary. This helps in making the dataset easier to work with.

### Approach:
- Use `df.rename(columns={})` to rename columns.
- Ensure all column names follow a consistent naming convention.
- Replace spaces or special characters in column names for compatibility.

In [29]:
print(f"Orginigal names : {df.columns}")
# Renaming columns for better readability/Standardizing column names
df.rename(columns={
    'Loan_ID': 'loan_id',
    'Gender': 'gender',
    'Married': 'married',
    'Dependents': 'dependents',
    'Education': 'education',
    'Self_Employed': 'self_employed',
    'ApplicantIncome': 'applicant_income',
    'CoapplicantIncome': 'coapplicant_income',
    'LoanAmount': 'loan_amount',
    'Loan_Amount_Term': 'loan_term',
    'Credit_History': 'credit_history',
    'Property_Area': 'property_area',
    'Loan_Status': 'loan_status'
}, inplace=True)

# Display updated column names
print("Updated column names")
df.columns

Orginigal names : Index(['loan_id', 'gender', 'married', 'dependents', 'education',
       'self_employed', 'applicant_income', 'coapplicant_income',
       'loan_amount', 'loan_term', 'credit_history', 'property_area',
       'loan_status'],
      dtype='object')
Updated column names


Index(['loan_id', 'gender', 'married', 'dependents', 'education',
       'self_employed', 'applicant_income', 'coapplicant_income',
       'loan_amount', 'loan_term', 'credit_history', 'property_area',
       'loan_status'],
      dtype='object')

## Binning/Discretization

Binning helps group continuous numerical values into categories. This is useful when analyzing ranges instead of individual values.

### Example:
- Categorizing loan amounts into "Low", "Medium", and "High" bins.

In [38]:
# Define bins and labels
bins = [0, 100000, 200000, 700000]  # Example loan amount ranges
labels = ['Low', 'Medium', 'High']

# Apply binning to Loan_Amount
df['loan_amount_category'] = pd.cut(df['loan_amount'], bins=bins, labels=labels)

# Display the first few rows to verify changes
print(df[['loan_amount', 'loan_amount_category']].head())
print("="*50)
print(df[['loan_amount', 'loan_amount_category']].tail())


   loan_amount loan_amount_category
0        135.0                  Low
1         98.0                  Low
2        350.0                  Low
3        113.0                  Low
4        168.0                  Low
     loan_amount loan_amount_category
609        120.0                  Low
610        600.0                  Low
611        700.0                  Low
612        490.0                  Low
613        360.0                  Low


## Handling Outliers

Outliers can distort the dataset and impact analysis. We can handle them by:
- Removing extreme values using statistical techniques.
- Using the interquartile range (IQR) method.

### Approach:
- Calculate IQR.
- Filter out values beyond the acceptable range.

In [41]:
# Handling outliers using the IQR method
Q1 = df['loan_amount'].quantile(0.25)
Q3 = df['loan_amount'].quantile(0.75)
IQR = Q3 - Q1

# Define the range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df = df[(df['loan_amount'] >= lower_bound) & (df['loan_amount'] <= upper_bound)]

# Display updated dataset shape
df.shape


(573, 14)

## Random Sampling

If working with large datasets, we may need to create a smaller subset for quick analysis.

### Approach: 1
- Use `df.sample(n=, random_state=)` to get a random subset.

In [48]:
# Select a random sample of 200 rows
df_sample = df.sample(n=200, random_state=42)

# Display sample size
df_sample.shape 

(200, 14)

### Approach: 2

In [55]:
# Select 10% of the  dataset randomly
sample_df = df.sample(frac=0.1, random_state=42)
print(f"Random sampling completed. Sample size: {len(sample_df)}")

sample_df

Random sampling completed. Sample size: 57


,loan_id,gender,married,dependents,education,self_employed,applicant_income,coapplicant_income,loan_amount,loan_term,credit_history,property_area,loan_status,loan_amount_category
524,LP001998,Male,Yes,2,Not Graduate,No,7667,0.0,185.0,360.0,1.0,Rural,Y,Low
71,LP001514,Female,Yes,0,Graduate,No,2330,4486.0,100.0,360.0,1.0,Semiurban,Y,Low
133,LP002714,Male,No,1,Not Graduate,No,2679,1302.0,94.0,360.0,1.0,Semiurban,Y,Low
408,LP001520,Male,Yes,0,Graduate,No,4860,830.0,125.0,360.0,1.0,Semiurban,Y,Low
563,LP002795,Male,Yes,3+,Graduate,Yes,10139,0.0,260.0,360.0,1.0,Semiurban,Y,Low
323,LP001914,Male,Yes,0,Graduate,No,3927,800.0,112.0,360.0,1.0,Semiurban,Y,Low
190,LP001846,Female,No,3+,Graduate,No,3083,0.0,255.0,360.0,1.0,Rural,Y,Low
30,LP002898,Male,Yes,1,Graduate,No,1880,0.0,61.0,360.0,1.0,Rural,N,Low
82,LP001123,Male,Yes,0,Graduate,No,2400,0.0,75.0,360.0,1.0,Urban,Y,Low
312,LP001720,Male,Yes,3+,Not Graduate,No,3850,983.0,100.0,360.0,1.0,Semiurban,Y,Low


## Creating Indicator/Dummy Variables

Machine learning models often require numerical input. We convert categorical variables into dummy variables using `pd.get_dummies()`.

### Example:
- Convert the "Married" column into numeric indicators.

In [61]:
df.columns

Index(['loan_id', 'gender', 'married', 'dependents', 'education',
       'self_employed', 'applicant_income', 'coapplicant_income',
       'loan_amount', 'loan_term', 'credit_history', 'property_area',
       'loan_status', 'loan_amount_category'],
      dtype='object')

In [ ]:
# Create dummy variables for categorical columns
df = pd.get_dummies(df, columns=['Married', 'Gender', 'Self_Employed'])    #Dummies have been created for thes three columns.

# Display the first few rows
df.head()

In [78]:
df.columns

Index(['loan_id', 'dependents', 'education', 'applicant_income',
       'coapplicant_income', 'loan_amount', 'loan_term', 'credit_history',
       'property_area', 'loan_status', 'loan_amount_category', 'married_No',
       'married_Yes', 'gender_Female', 'gender_Male', 'self_employed_No',
       'self_employed_Yes'],
      dtype='object')

In [80]:
# Create dummy variables for categorical columns
df = pd.get_dummies(df, columns=['loan_amount_category']) #Dummies have been created for this column.

In [82]:
df.columns

Index(['loan_id', 'dependents', 'education', 'applicant_income',
       'coapplicant_income', 'loan_amount', 'loan_term', 'credit_history',
       'property_area', 'loan_status', 'married_No', 'married_Yes',
       'gender_Female', 'gender_Male', 'self_employed_No', 'self_employed_Yes',
       'loan_amount_category_Low', 'loan_amount_category_Medium',
       'loan_amount_category_High'],
      dtype='object')

In [84]:
df.head()

,loan_id,dependents,education,applicant_income,coapplicant_income,loan_amount,loan_term,credit_history,property_area,loan_status,married_No,married_Yes,gender_Female,gender_Male,self_employed_No,self_employed_Yes,loan_amount_category_Low,loan_amount_category_Medium,loan_amount_category_High
0,LP001722,0,Graduate,150,1800.0,135.0,360.0,1.0,Rural,N,False,True,False,True,True,False,True,False,False
1,LP002502,2,Not Graduate,210,2917.0,98.0,360.0,1.0,Semiurban,Y,False,True,True,False,True,False,True,False,False
3,LP002603,0,Graduate,645,3683.0,113.0,480.0,1.0,Rural,Y,True,False,True,False,True,False,True,False,False
4,LP001644,0,Graduate,674,5296.0,168.0,360.0,1.0,Rural,Y,False,True,False,True,False,True,True,False,False
5,LP001259,1,Graduate,1000,3022.0,110.0,360.0,1.0,Urban,N,False,True,False,True,False,True,True,False,False


## Applying Functions

We can apply custom transformations to columns using the `apply()` method. This allows for complex modifications, such as converting text data or creating new derived columns.

### Example:
- Converting loan amounts from thousands to actual amounts.
- Creating a new column to indicate whether an applicant has dependents.

In [ ]:
# Converting loan amount from thousands to actual numbers
df['Loan_Amount'] = df['Loan_Amount'].apply(lambda x: x * 1000)

# Creating a new column indicating if the applicant has dependents
df['Has_Dependents'] = df['Dependents'].apply(lambda x: 'Yes' if x != '0' else 'No')

# Display the first few rows to verify changes
df[['Loan_Amount', 'Has_Dependents']].head()

# Step 5 & 6: String Manipulation & Handling Categorical Data

In this step, we will:
1. **Clean and standardize string-based categorical values** to ensure consistency.
2. **Convert categorical data into an optimized format** for better memory efficiency.

## Tasks:
- **Step 5: String Manipulation**
  - Trim whitespace and ensure uniform case formatting.
  - Handle special characters if present.
  - Replace inconsistent values.

- **Step 6: Handling Categorical Data**
  - Convert categorical columns to the `category` data type.
  - Display memory usage before and after optimization.

In [90]:
# Standardizing string values (trimming whitespace & converting to lowercase)
string_cols = ['education', 'property_area', 'loan_status']

for col in string_cols:
    df[col] = df[col].str.strip().str.lower()

# Verify changes
df[string_cols].head()


,education,property_area,loan_status
0,graduate,rural,n
1,not graduate,semiurban,y
3,graduate,rural,y
4,graduate,rural,y
5,graduate,urban,n


In [98]:
# Display memory usage before optimization
print("Memory usage before optimization:")
print(df.memory_usage(deep=True))

# Convert categorical columns to 'category' dtype for efficiency
categorical_cols = ['education', 'property_area', 'loan_status', 'credit_history', 'dependents']
for col in categorical_cols:
    df[col] = df[col].astype('category')

# Display memory usage after optimization
print("\nMemory usage after optimization:")
print(df.memory_usage(deep=True))

# Verify column data types
df.dtypes

Memory usage before optimization:
Index                           4584
loan_id                        32661
dependents                     28690
education                        799
applicant_income                4584
coapplicant_income              4584
loan_amount                     4584
loan_term                       4584
credit_history                  4584
property_area                  31822
loan_status                    28650
married_No                       573
married_Yes                      573
gender_Female                    573
gender_Male                      573
self_employed_No                 573
self_employed_Yes                573
loan_amount_category_Low         573
loan_amount_category_Medium      573
loan_amount_category_High        573
dtype: int64

Memory usage after optimization:
Index                           4584
loan_id                        32661
dependents                       946
education                        799
applicant_income               

loan_id                          object
dependents                     category
education                      category
applicant_income                  int64
coapplicant_income              float64
loan_amount                     float64
loan_term                       float64
credit_history                 category
property_area                  category
loan_status                    category
married_No                         bool
married_Yes                        bool
gender_Female                      bool
gender_Male                        bool
self_employed_No                   bool
self_employed_Yes                  bool
loan_amount_category_Low           bool
loan_amount_category_Medium        bool
loan_amount_category_High          bool
dtype: object

In [100]:
df.to_csv('loandata_cleaned.csv', index=False)